# API: Particles and Pushers

Struphy provides a convenient interface to [particle data structures](https://struphy.pages.mpcdf.de/struphy/tutorials/tutorial_06_data_structures.html#PIC-data-structures) and [particle pushers](https://struphy.pages.mpcdf.de/struphy/sections/subsections/pic_base.html#module-struphy.pic.pushing.pusher) (ODE solvers) for trying out new particle schemes.
Check out the documentation for more details.

In [ ]:
from struphy.pic.particles import Particles3D
from struphy.feec.psydac_derham import Derham
from mpi4py import MPI

Nel = [8, 8, 12]  # number of elements
p = [2, 3, 4]  # spline degrees
spl_kind = [False, False, True] # spline boundary conditions
comm = MPI.COMM_WORLD

derham = Derham(Nel, p, spl_kind, comm=comm)

particles = Particles3D('ions', derham)

particles.draw_markers()

In [ ]:
particles.markers

In [ ]:
from struphy.pic.pushing.pusher import Pusher, ButcherTableau
from struphy.pic.pushing.pusher_kernels import push_random_diffusion_stage
from numpy import random, array

noise = array(particles.markers[:, :3])
diffusion_coeff = 1.
butcher = ButcherTableau('forward_euler')

args_kernel = (noise,
               diffusion_coeff,
               butcher.a,
               butcher.b,
               butcher.c)

pusher = Pusher(particles, 
                push_random_diffusion_stage, 
                args_kernel,
                derham.args_derham, 
                particles.domain.args_domain,
                alpha_in_kernel=1.)

In [ ]:
mean = [0, 0, 0]
cov = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
noise[:] = random.multivariate_normal(mean, cov, len(particles.markers))

dt = 0.1
pusher(dt)